# Hosting Detectron2 model on Sagemaker Inference endpoint

In this notebook we'll package previously trained model into PyTorch Serving container and deploy it on Sagemaker. First, let's review serving container. There are two key difference comparing to training container:
- we are using different base container provided by Sagemaker;
- we need to start Web server (refer to ENTRYPOINT command).

## Compiling Serving Container

In [ ]:
! pygmentize -l docker Dockerfile.serving

As in case of training image, we'll need to build and push container to AWS ECR. Before this, we'll need to loging to shared Sagemaker ECR and your local ECR

In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-2.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 553020858742.dkr.ecr.us-east-2.amazonaws.com

Now, let's build and push container using follow command. Note, that here we supply non-default Dockerfile.

In [ ]:
! ./build_and_push.sh d2-sm-coco-serving latest Dockerfile.serving

## Preparing test data

We'll be using coco2017 validation dataset. To simplify working with it, let's install locally Pycoco package

In [ ]:
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!pip install -U scikit-image

In [ ]:
!pip install google-colab # needed for image rendering

Now, let's download Coco2017 validation dataset

In [ ]:
data_dir = "../datasets/coco/" # folder where data will be saved
dataset  = "val2017"

In [ ]:
! mkdir -p {data_dir}{dataset}
! wget http://images.cocodataset.org/zips/val2017.zip -P {data_dir}
! unzip {data_dir}/val2017.zip -d {data_dir}

In [ ]:
! wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P {data_dir}
! unzip {data_dir}/annotations_trainval2017.zip -d {data_dir}{dataset}

Let's get a random image ...

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

annFile='{}{}/annotations/instances_{}.json'.format(data_dir,dataset, dataset)
coco=COCO(annFile)

# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','dog']);
imgIds = coco.getImgIds(catIds=catIds);
imgId = imgIds[np.random.randint(len(imgIds))]
image_instance = coco.loadImgs(imgId)[0]
image_np = io.imread(image_instance['coco_url'])    

In [ ]:
plt.axis('off')
plt.imshow(image_np)
plt.show()

# Testing inference script locally

Let's first check what inference script we'll deploy:

In [ ]:
!pygmentize container_serving/predict_coco.py

To test inference pipeline locally, you can run your `container_serving/predict_coco.py` locally (only code in __main__ guard will be executed). You'll need to have Detectron2 and number other packages locally installed to test it.

Make sure that you pass correct --model-dir argument.

In [ ]:
!python container_serving/predict_coco.py --image container_serving/coco_sample.jpg --model-dir ../trained_model

# Deploying Inference Endpoint

Below is some initial imports and configuration.

In [7]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import requests
import json
import pickle
import detectron2 

role = get_execution_role()

In [8]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

In [9]:
print(sess)

## Define parameters of your container

In [4]:
container_serving = "d2-sm-coco-serving" # your container name
tag = "latest" # you can have several version of container available
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container_serving, tag)

print("Following container will be used for hosting: ",image)

Following container will be used for hosting:  553020858742.dkr.ecr.us-east-2.amazonaws.com/d2-sm-coco-serving:latest


## Deploy endpoint locally

As training on COCO2017 can be quite lenghty, we'll deploy our endpoint from model artifacts from already completed training jobs. Please review your training jobs, and find one which succesffuly completed. Then, copy model artifact S3 URI and.  pass it to `model_data` argument below.

In [ ]:
from sagemaker.pytorch import PyTorchModel, PyTorch, PyTorchPredictor
from sagemaker import Model


model = PyTorchModel(
                     model_data="s3://sagemaker-us-east-2-553020858742/detectron2-model/R101-FPN.tar.gz",
                     role=role,
                     entry_point="predict_coco.py", source_dir="container_serving",
                     framework_version="1.4", py_version="3.6",
                     image=image)

In [ ]:
predictor = model.deploy(
                         instance_type = 'local_gpu',
                         initial_instance_count=1,
                         endpoint_name=f"{container_serving}-{tag}", # define a unqie endpoint name; if ommited, Sagemaker will generate it based on used container
                         tags=[{"Key":"image", "Value":f"{container_serving}:{tag}"}], 
                         wait=False
                         )

In [ ]:
url = 'http://172.18.0.2:8080/invocations' # IP address of serving container when deployed locally
content_type = 'jpeg'
accept_type = "detectron2" # "json" or "detectron2" supported
headers = {'content-type': content_type, 'accept': accept_type}
payload = open('coco_sample.jpg', 'rb')

response = requests.post(url, data=payload, headers=headers)

if accept_type=="json":
    predictions = json.loads(response.content)
elif accept_type=="detectron2":
    predictions = pickle.loads(response.content)

print(predictions)

## Deploy remote endpoint

To process inference data when we are sending it over internet, we need to have two customer ser/deser methods.

In [19]:
from sagemaker.pytorch import PyTorchModel, PyTorch, PyTorchPredictor
from sagemaker.estimator import Estimator, Model

remote_model = PyTorchModel(
                     name = "d2-service-v7", 
                     model_data="s3://sagemaker-us-east-2-553020858742/detectron2-model/R101-FPN-v2.tar.gz", # from training job
                     role=role,
                     entry_point="predict_coco.py", source_dir="container_serving",
                     framework_version="1.4", py_version="3.6",
                     image=image)

In [20]:
endpoint_name = f"{container_serving}-{tag}"

remote_predictor = remote_model.deploy(
                         instance_type='ml.p3.16xlarge', 
                         initial_instance_count=1,
                         endpoint_name=endpoint_name+"v7", # define a unqie endpoint name; if ommited, Sagemaker will generate it based on used container
                         tags=[{"Key":"image", "Value":f"{container_serving}:{tag}"}], 
                         wait=False
                         )

In [18]:
import boto3
import skimage.io
from io import BytesIO
#import d2_deserializer

pred_image = 'coco_sample.jpg'

client = boto3.client('sagemaker-runtime')
accept_type = "json"
content_type = 'image/jpeg'
headers = {'content-type': content_type}
payload = open(pred_image, 'rb')
image_np = skimage.io.imread(pred_image)
height, weight, _ = image_np.shape
print(image_np.shape)
endpoint_name = "d2-sm-coco-serving-latestv6"

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType=content_type,
    Accept = accept_type
)

if accept_type=="json":
    predictions = json.loads(response['Body'].read())
elif accept_type=="detectron2":
    print(response['Body'].read())
    stream = BytesIO(response['Body'].read())
    predictions = pickle.loads(stream.read())
    
print(predictions)


(480, 640, 3)
{'scores': [0.9974532723426819, 0.9938008189201355, 0.988240122795105, 0.9850716590881348, 0.9844247102737427, 0.9810763597488403, 0.9800938963890076, 0.9720492362976074, 0.9688801765441895, 0.9674813747406006, 0.927189290523529, 0.9221163988113403, 0.9185774326324463, 0.9142449498176575, 0.8913487792015076, 0.8826121687889099, 0.8605493903160095, 0.8423078656196594, 0.8416911363601685, 0.8005133271217346, 0.7691927552223206, 0.7283533811569214, 0.7125754356384277, 0.6947720050811768, 0.6323946118354797, 0.5554373264312744, 0.502210259437561], 'pred_classes': [0, 2, 2, 2, 9, 0, 2, 0, 2, 0, 0, 5, 24, 9, 2, 2, 9, 5, 9, 2, 2, 0, 2, 0, 9, 9, 9], 'pred_boxes': [[309.5509033203125, 163.2444610595703, 434.3181457519531, 475.4096984863281], [3.7802395820617676, 242.15467834472656, 537.434814453125, 432.81524658203125], [14.279938697814941, 212.42967224121094, 45.97575759887695, 233.4514617919922], [421.2859191894531, 208.60498046875, 556.5548706054688, 309.9937744140625], [143.99

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
r = register_coco_instances("coco2017", {}, "json_annotation_val.json", "../datasets/coco/val2017")
metadata = MetadataCatalog.get("coco2017")

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg

In [ ]:
from detectron2.utils.visualizer import ColorMode, Visualizer
import cv2
from google.colab.patches import cv2_imshow


im = cv2.imread('coco_sample.jpg')
v = Visualizer(im[:, :, ::-1],
               metadata=MetadataCatalog.get("coco_2017_train"), 
               scale=1.2, 
)
v = v.draw_instance_predictions(predictions["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
predictor.delete_endpoint()